In [1]:
import numpy as np
from sequence_parser.sequence import Sequence
from sequence_parser.port import Port
from sequence_parser.circuit import Circuit
from sequence_parser.backend import PortTable, GateTable, Backend
from sequence_parser.instruction import *
import qcodes as qc
from qcodes_drivers.N5222A import N5222A
from plottr.data.datadict_storage import DataDict, DDH5Writer
from sequence_parser.variable import Variable, Variables

In [2]:
station = qc.Station()
vna = N5222A("vna", "TCPIP0::192.168.100.17::inst0::INSTR")
vna.electrical_delay(42e-9)  # s
vna.meas_trigger_input_type("level")
vna.meas_trigger_input_polarity("positive")
vna.aux1.output_polarity("negative")
vna.aux1.output_position("after")
#vna.aux1.trigger_mode("point")
station.add_component(vna)

'vna'

In [3]:
def run_drive_sweep():
    vna.output(True)
 #   drive_source.output(True)
 #   drive_source.start_sweep()
    vna.sweep_mode("single")
    try:
        while not (vna.done()):# and drive_source.sweep_done()):
            #if (vna.done() or drive_source.sweep_done()):
            #    print(vna.done(), drive_source.sweep_done())
            time.sleep(0.2)
    finally:
        vna.output(False)
        #drive_source.output(False)

In [4]:
probe_freq = Variable(name="probe_freq", value_array=np.linspace(6.73, 6.77, 41)*10e9, unit="Hz")
probe_amp = Variable(name="probe_amp", value_array=np.linspace(-10, 0, 11), unit="dbm")
#pump_freq = Variable(name="resonator_drive", value_array=np.linspace(6.73, 6.77, 41)*10e9, unit="Hz")
var = Variables()
var.add(probe_amp)
var.add(probe_freq)

var.compile()

readout_port = Port("probe_port", 0, max_amp=20)

In [5]:
readout_pulse = Square(amplitude=probe_amp, duration=1)

In [6]:
onetone_seq = Sequence([readout_port])
onetone_seq.add(SetDetuning(probe_freq),readout_port)
onetone_seq.add(readout_pulse, readout_port, copy=False)

In [7]:
for update_command in var.update_command_list:
    onetone_seq.update_variables(update_command)
    onetone_seq.compile()
    print(onetone_seq.port_list[0].detuning)
    print(abs(onetone_seq.port_list[0].waveform[0]))

67300000000.00001
10.0
67310000000.00001
10.0
67320000000.0
10.0
67330000000.00001
10.0
67340000000.0
10.0
67350000000.0
10.0
67360000000.00001
10.0
67370000000.0
10.0
67380000000.00001
10.0
67390000000.0
10.0
67400000000.0
10.0
67410000000.00001
10.0
67420000000.0
10.0
67430000000.0
10.0
67440000000.0
10.0
67450000000.0
10.0
67460000000.00001
10.0
67470000000.0
10.0
67480000000.0
10.0
67490000000.0
10.0
67500000000.0
10.0
67510000000.0
10.0
67520000000.0
10.0
67530000000.0
10.0
67539999999.99999
10.0
67550000000.0
10.0
67560000000.0
10.0
67570000000.0
10.0
67580000000.0
10.0
67589999999.99999
10.0
67600000000.0
10.0
67610000000.0
10.0
67619999999.99999
10.0
67630000000.0
10.0
67639999999.99999
10.0
67650000000.0
10.0
67660000000.0
10.0
67669999999.99999
10.0
67680000000.0
10.0
67689999999.99999
10.0
67699999999.99999
10.0
67300000000.00001
9.0
67310000000.00001
9.0
67320000000.0
9.0
67330000000.00001
9.0
67340000000.0
9.0
67350000000.0
9.0
67360000000.00001
9.0
67370000000.0
9.0
67380

array([-10.        +0.j        ,  -9.99999999-0.00048365j,
        -9.99999995-0.00096729j,  -9.99999996-0.00084059j,
        -9.99999981-0.00193459j,  -9.99999954-0.00302858j,
        -9.99999986-0.00168118j,  -9.99999864-0.00521658j,
        -9.99999925-0.00386917j,  -9.99999968-0.00252176j])